In [1]:
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# Load all songs
songs = []
for f in os.listdir("data/preprocessed"):
    songs.append(np.genfromtxt(("data/preprocessed/%s" % f), dtype=int, delimiter=','))

In [3]:
# Split data up into "patterns"
pattern_length = 200
data_X = []
data_y = []
for f in songs:
    for i in range(0, len(f) - pattern_length, 1):
        data_X.append(f[i:i+pattern_length])
        data_y.append(f[i+pattern_length])
n_patterns = len(data_X)
print("Total Patterns: ", n_patterns)

Total Patterns:  105088


In [4]:
# Find unused notes
freq = np.bincount(np.array(data_X).flatten())

# Get indices of nonzero frequencies
non_zero_freq = np.nonzero(freq)[0]

# Remember highest and lowest used notes
lowest = non_zero_freq[0]
highest = non_zero_freq[len(non_zero_freq) - 1]
n_notes = highest - lowest + 1

In [5]:
# Reshape X
X = np.reshape(data_X, (n_patterns, pattern_length, 1))

# Normalize
X = (X - lowest) / n_notes

# One hot encode
y = np_utils.to_categorical(data_y - lowest)

In [6]:
# LSTM
model = Sequential()
model.add(LSTM(512, input_shape=((pattern_length, 1)), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(n_notes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [7]:
# Checkpoints
checkpoint = ModelCheckpoint("checkpoint-{epoch:02d}.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)

In [114]:
model.fit(X, y, epochs=40, batch_size=64, callbacks=[checkpoint])

Epoch 1/40
102414/102414 [==============================] - 9918s 97ms/step - loss: 2.8566 - categorical_accuracy: 0.4037

Epoch 00001: loss improved from inf to 2.85665, saving model to checkpoint-01.hdf5
Epoch 2/40
102414/102414 [==============================] - 9861s 96ms/step - loss: 2.6801 - categorical_accuracy: 0.4044

Epoch 00002: loss improved from 2.85665 to 2.68013, saving model to checkpoint-02.hdf5
Epoch 3/40
102414/102414 [==============================] - 9792s 96ms/step - loss: 2.3307 - categorical_accuracy: 0.4141

Epoch 00003: loss improved from 2.68013 to 2.33073, saving model to checkpoint-03.hdf5
Epoch 4/40
102414/102414 [==============================] - 9794s 96ms/step - loss: 2.2231 - categorical_accuracy: 0.4222

Epoch 00004: loss improved from 2.33073 to 2.22309, saving model to checkpoint-04.hdf5
Epoch 5/40
102414/102414 [==============================] - 9871s 96ms/step - loss: 2.1561 - categorical_accuracy: 0.4281

Epoch 00005: loss improved from 2.22309 t

KeyboardInterrupt: 

In [19]:
# Load 
filename = "checkpoint-17.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [52]:
start = np.random.randint(0, len(data_X) - 1)
pattern=X[start]
song = []
#Generate
for i in range(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0).flatten()
    index=np.argmax(prediction)
    pattern = np.append(pattern, (index/float(n_notes)))
    pattern = pattern[1:len(pattern)]
    song = np.append(song, (index))

In [53]:
print((pattern*n_notes) + lowest)
np.savetxt("data/song.csv", (song + lowest), fmt='%s', delimiter=",")

[ 77.  77. 128.  72.  65.  77.  65.  65.  77. 128.  65.  65.  70.  65.
  65.  72. 128.  65.  60.  72.  60.  60.  72. 128.  65.  60.  70.  60.
  60.  72. 128.  65.  60.  70.  60.  60.  72. 128.  65.  60.  70.  60.
  65.  72. 128.  65.  65.  72.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72.  77. 128.  72.  65.  77.  65.  72.  77. 128.  72.  65.  77.  65.
  72. 